In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///./data/databases/chinook.db')

In [3]:
tracks = pd.read_sql('tracks', con=engine)

In [4]:
tracks.head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
engine.table_names()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

In [6]:
invoice_items = pd.read_sql('invoice_items', con=engine)
invoice_items.head()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [8]:
invoice_items.groupby('TrackId').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)

,Quantity
TrackId,
3500,2
2263,2
2945,2
2946,2
1389,2
...,...
1215,1
1214,1
1213,1


In [10]:
tracks.loc[tracks['TrackId'] == 3500]

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99


In [11]:
playlist = pd.read_sql('playlists', con=engine)
playlist.head()

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


In [12]:
pt = pd.read_sql('playlist_track', con=engine)
pt.head()

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


In [13]:
pt.groupby('PlaylistId').size()

PlaylistId
1     3290
3      213
5     1477
8     3290
9        1
10     213
11      39
12      75
13      25
14      25
15      25
16      15
17      26
18       1
dtype: int64

In [15]:
pt_15 = pt.groupby('PlaylistId').filter(lambda x: len(x) >= 15)
pt_15.head()

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


In [16]:
pt_15['PlaylistId'].value_counts()

1     3290
8     3290
5     1477
3      213
10     213
12      75
11      39
17      26
15      25
14      25
13      25
16      15
Name: PlaylistId, dtype: int64

In [19]:
pt_15.merge(tracks).groupby('PlaylistId').agg({'UnitPrice': 'mean'}).sort_values('UnitPrice', ascending=False)

,UnitPrice
PlaylistId,
3,1.99
10,1.99
5,0.99
16,0.99
11,0.99
12,0.99
13,0.99
14,0.99
15,0.99


In [20]:
artists = pd.read_sql('artists', con=engine)
artists.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [21]:
albums = pd.read_sql('albums', con=engine)
albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [36]:
aa = artists.merge(albums)
aa.head()

,ArtistId,Name,AlbumId,Title
0,1,AC/DC,1,For Those About To Rock We Salute You
1,1,AC/DC,4,Let There Be Rock
2,2,Accept,2,Balls to the Wall
3,2,Accept,3,Restless and Wild
4,3,Aerosmith,5,Big Ones


In [37]:
aat = tracks.merge(aa, on='AlbumId')

In [38]:
aat.head()

,TrackId,Name_x,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,ArtistId,Name_y,Title
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,1,AC/DC,For Those About To Rock We Salute You
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,1,AC/DC,For Those About To Rock We Salute You
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,1,AC/DC,For Those About To Rock We Salute You
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,1,AC/DC,For Those About To Rock We Salute You
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,1,AC/DC,For Those About To Rock We Salute You


In [40]:
grp = aat.groupby('ArtistId').agg({'Milliseconds': 'mean'}).reset_index()

In [43]:
grp['Milliseconds'] = pd.to_datetime(grp['Milliseconds'])

In [44]:
grp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ArtistId      204 non-null    int64         
 1   Milliseconds  204 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.3 KB


In [28]:
grp['Milliseconds'].dt.

(347, 4)